In [1]:
# prep for notebook
import sys
from pathlib import Path

# 添加项目根目录到路径
notebook_dir = Path().resolve()     # test 目录
project_root = notebook_dir.parent  # 项目根目录
sys.path.insert(0, str(project_root))

import os
os.chdir('../')

In [2]:
import re
from pprint import pprint
import module.config as config
import module.utils as utils

In [3]:
utils.download_files()

ConnectTimeout: HTTPSConnectionPool(host='ini.42kit.com', port=443): Max retries exceeded with url: /full/global.ini (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B581FAAD10>, 'Connection to ini.42kit.com timed out. (connect timeout=None)'))

In [ ]:
def text_proc(s: str):
    tbremoved = [' ', '\xa0']
    for chr in tbremoved:
        s = re.sub(f"^{chr}*", '', s)
        s = re.sub(f"{chr}*$", '', s)
    return s.replace('\u00a0', ' ')

In [ ]:
reader = utils.TextReader()
ids = reader.find_ids_by_pattern(r'^item_desc.*$', True)
raw_data = dict()
raw_set = set()
for id in ids:
    texts = reader.get(id)
    if texts is None or texts['cn'] is None or texts['en'] is None: continue
    if '：' not in texts['cn'] or ': ' not in texts['en']: continue
    lines = []
    for i in range(len(texts['cn'].split('\\n'))):
        if len(texts['cn']) and len(texts['en']):
            lines.append((
                text_proc(texts['cn'].split('\\n')[i]), 
                text_proc(texts['en'].split('\\n')[i])
            ))
    if len(lines): 
        raw_data[id] = lines
        raw_set.update(lines)
    

In [ ]:
effects       = set()
manufacturers = set()

for pair in raw_set:
    t_cn, t_en = pair
    t_cn, t_en = str(t_cn), str(t_en)
    if t_cn.count('：') != 1 or t_en.count(': ') != 1: continue
    if 'Effects' not in t_en and 'Manufacturer' not in t_en:
        continue
    if 'Effects' in t_en:
        effects.add(pair)
    elif 'Manufacturer' in t_en:
        manufacturers.add(pair)
        
for data, file_path in [
    (effects, 'data/effects_gen.map'),
    (manufacturers, 'data/manufacturers_gen.map')
]:
    data = sorted(list(data))
    write_data = set()
    for d in data:
        t_cn, t_en = d
        write_data.add(f"{t_en[t_en.index(': ')+2:]}={t_cn[t_cn.index('：')+1:]}\n")
        
    write_data = sorted(list(write_data))
    with open(file_path, 'w', -1, config.ENCODE) as file:
        file.writelines(write_data)


In [6]:
with open('data\manufacturers_gen_nosuffix.map', 'r', -1, config.ENCODE) as file:
    manufacturers = set(file.readlines())

with open('data\manufacturers.map', 'w', -1, config.ENCODE) as file:
    file.writelines(sorted(manufacturers))

In [ ]:
tkmap = set()
tvmap = set()
tklist = []
tvlist = []
for tid, lines in raw_data.items():
    for t_cn, t_en in lines:
        t_cn, t_en = str(t_cn), str(t_en)
        if t_cn.count('：') != 1 or t_en.count(': ') != 1: continue
        ck, _, cv = t_cn.partition('：')
        ek, _, ev = t_en.partition(': ')
        cv, ev = re.sub(r'\d{1,}', '[NUM]', cv), re.sub(r'\d{1,}', '[NUM]', ev)
        tkmap.add((ek, ck))
        tvmap.add((ev, cv))
        tklist.append((ek, ck))
        tvlist.append((ev, cv))

tkmap = sorted(list(tkmap))
tvmap = sorted(list(tvmap))
tklist.sort()
tvlist.sort()

count = 0

with open('data/ignore.ini', 'r', -1, config.ENCODE) as file:
    ignores = [l.replace('\n', '') for l in file.readlines()]

for d, s, f in [(tkmap, tklist, 'keys.ini'), (tvmap, tvlist, 'vals.ini')]:
    with open(f"data/map_{f}", 'w', -1, config.ENCODE) as file1:
        with open(f"data/count_{f}", 'w', -1, config.ENCODE) as file2:
            prev = ''
            for t_en, t_cn in d:
                if t_en in ignores: continue
                if prev != t_en and s.index((t_en, t_cn)) != 0:
                    file1.write("\n")
                    file2.write("\n")
                file1.write(f"{t_en}={t_cn}\n")
                file2.write(f"{s.count((t_en, t_cn)):<5d}| {t_en}={t_cn}\n")
                prev = t_en
                count = max(count, s.count((t_en, t_cn)))

print(count)

1955


In [57]:
keys = []
with open('data/keys.map', 'r', -1, config.ENCODE) as file:
    cache = [l.replace('\n', '') for l in file.readlines()]
    for keymap in cache:
        t_en, _, t_cn = keymap.partition('=')
        keys.append(t_en)

In [ ]:
src  = open('data/vals_raw.map', 'r', -1, config.ENCODE)
name = open('data/vals_name.map', 'w', -1, config.ENCODE)
num  = open('data/vals_num.map', 'w', -1, config.ENCODE)

for line in src.readlines():
    if '[NUM]' in line:
        num.write(line)
    else:
        name.write(line)

src.close()
name.close()
num.close()